In [1]:
import os
import sys
import yaml
import glob
from copy import deepcopy
from shutil import copy
from typing import (
    Dict,
    List, 
    Optional,
    Tuple,
    Union
)

In [2]:
# os.getcwd()

In [3]:
__file__ = os.path.join(os.getcwd(),"cs_wrap2.ipynb")

In [4]:
mod_path = os.path.join(os.path.dirname(os.path.abspath(__file__)),"..")

In [5]:
sys.path.append(mod_path)

In [6]:
from convert_source.cs_utils.bids_info import (
    construct_bids_dict,
    construct_bids_name,
    search_bids
)

In [7]:
from convert_source.cs_utils.utils import (
    BIDSimg,
    collect_info,
    convert_image_data,
    dict_multi_update,
    get_metadata,
    list_in_substr,
    SubInfoError,
    SubDataInfo,
    read_json,
    write_josn,
    # new functions
    comp_dict,
    depth,
    list_dict,
    get_bvals,
    gzip_file,
    gunzip_file,
    header_search
)

In [8]:
from convert_source.cs_utils.fileio import(
    LogFile,
    TmpDir,
    ConversionError,
    NiiFile
)

In [9]:
from convert_source.cs_utils.const import(
    DEFAULT_CONFIG,
    BIDS_PARAM
)

In [10]:
from convert_source.imgio.dcmio import (
    is_valid_dcm
)

In [11]:
from convert_source.imgio.niio import (
    get_data_params,
    get_num_frames
)

In [12]:
# test_config:
test_config = os.path.abspath(
    os.path.join(
        __file__,
        '..',
        '..',
        'notebooks.old',
        'config.test',
        'config.default.yml'
        ))
test_config

'/Users/adebayobraimah/Desktop/projects/convert_source/notebooks.old/config.test/config.default.yml'

In [13]:
# 1. read config file
# 2. check bids search and map dicts
# 3. collect subject directory information [sub/ses ID, and data paths], exclude data as necessary
# 4. pass to function: List[SubDataInfo], search_dict, bids_search, bids_map, and meta_dict

In [14]:
def read_config(config_file: Optional[str] = "", 
                verbose: Optional[bool] = False
                ) -> Tuple[Dict[str,str],Dict,Dict,Dict,List[str]]:
    '''Reads configuration file and creates a dictionary of search terms for 
    each modality provided that each BIDS modality is used as a key via the 
    keyword 'modality_search'. Should BIDS related parameter descriptions need 
    to be used when renaming files, the related search and mapping terms can included 
    via the keywords 'bids_search' and 'bids_map', respectively. If these keywords 
    are not specified, then empty dictionaries are returned. Should exclusions be provided 
    (via the key 'exclude') then an exclusion list is created. Should this not be provided, 
    then an empty list is returned.

    BIDS modalities:
        - anat:
            - T1w, T2w, FLAIR, etc.
        - func:
            - bold
                - task:
                    - resting state, <task-name>
        - dwi
        - fmap

    Usage example:
        >>> [search_dict, bids_search, bids_map, meta_dict, exclusion_list] = read_config(config_file)
    
    Arguments:
        config_file: File path to yaml configuration file. If no file is used, then the default configuration file is used.
        verbose: Prints additional information to screen.
    
    Returns: 
        Tuple of dictionaries and a list that consists of:
            * search_dict: Nested dictionary of heuristic modality search terms for BIDS modalities.
            * bids_search: Nested dictionary of heuristic BIDS search terms.
            * bids_map: Corresponding nested dictionary of BIDS mapping terms to rename files to.
            * meta_dict: Nested dictionary of metadata terms to write to JSON file(s).
            * exclusion_list: List of exclusion terms.
    
    Raises:
        ConfigFileReadError: Error that arises if no heuristic search terms are provided.
    '''
    class ConfigFileReadError(Exception):
        pass

    if config_file:
        config_file: str = os.path.abspath(config_file)
    else:
        config_file: str = DEFAULT_CONFIG

    with open(config_file) as file:
        data_map: Dict[str,str] = yaml.safe_load(file)
        if verbose:
            print("Initialized parameters from configuration file")
    
    # Required modality search terms
    if any("modality_search" in data_map for element in data_map):
        if verbose:
            print("Categorizing search terms")
        search_dict: Dict[str,str] = data_map["modality_search"]
        del data_map["modality_search"]
    else:
        if verbose:
            print("Heuristic search terms required. Exiting...")
        raise ConfigFileReadError("Heuristic search terms required. Exiting...")
    
    # BIDS search terms
    if any("bids_search" in data_map for element in data_map):
        if verbose:
            print("Including BIDS related search term settings")
        bids_search: Dict[str,str] = data_map["bids_search"]
        del data_map["bids_search"]
    else:
        if verbose:
            print("No BIDS related search term settings")
        meta_dict: Dict = dict()
    
    # BIDS mapping terms
    if any("bids_map" in data_map for element in data_map):
        if verbose:
            print("Corresponding BIDS mapping settings")
        bids_map: Dict[str,str] = data_map["bids_map"]
        del data_map["bids_map"]
    else:
        if verbose:
            print("No BIDS mapping settings")
        meta_dict: Dict = dict()
    
    # Metadata terms
    if any("metadata" in data_map for element in data_map):
        if verbose:
            print("Including additional settings for metadata")
        meta_dict: Dict[str,Union[str,int]] = data_map["metadata"]
        del data_map["metadata"]
    else:
        if verbose:
            print("No metadata settings")
        meta_dict: Dict = dict()
    
    # Exclusion terms  
    if any("exclude" in data_map for element in data_map):
        if verbose:
            print("Exclusion option implemented")
        exclusion_list: List[str] = data_map["exclude"]
        del data_map["exclude"]
    else:
        if verbose:
            print("Exclusion option not implemented")
        exclusion_list: List = list()
        
    return (search_dict,
            bids_search,
            bids_map,
            meta_dict,
            exclusion_list)

In [15]:
def bids_id(s:str,
            search_dict: Dict,
            bids_search: Optional[Dict] = None,
            bids_map: Optional[Dict] = None,
            bids_name_dict: Optional[Dict] = None,
            parent_dir: Optional[str] = "",
            modality_type: Optional[str] = "",
            modality_label: Optional[str] = "",
            task: Optional[str] = ""
           ) -> Tuple[Dict[str,str],str,str,str]:
    '''
    TODO:
        * write documentation.
        
    Performs identification of descriptive BIDS information relevant for file naming, provided
    a BIDS search dictionary and a BIDS map dictionary. The resulting information is then placed
    in nested dictionary of BIDS related descriptive terms.
    
    Usage example:
        >>> bids_example_dict = bids_id("ImageFile00001.dcm")
        
    Arguments:
        s: Input str/file to be searched.
        search_dict: Dictionary of modality specific search terms.
        bids_search: Dictionary of BIDS specific search terms.
        bids_map: Dictionary of BIDS related terms to be mapped to BIDS search terms.
        bids_name_dict: Existing BIDS name dictionary. If provided, an updated copy of this dictionary is returned.
        parent_dir: Parent or study image directory of the image file. This path is removed from the string search
            to prevent false positives.
        modality_type: (BIDS) modality type (e.g. 'anat', 'func', etc).
        modality_label: (BIDS) modality label (e.g. 'T1w', 'bold', etc).
        task: (BIDS) task label (e.g. 'rest','nback', etc).
        
    Returns:
        Tuple that consists of:
            * Nested dictionary of BIDS descriptive naming related terms.
            * Modality type.
            * Modality label.
            * Task label.
    '''
    search_arr: List[str] = list_dict(d=search_dict)
    
    if os.path.exists(s) and parent_dir:
        if 'windows' in platform.platform().lower():
            path_sep: str = "\\"
        else:
            path_sep: str = "/"
        
        # Store string, then overwrite
        img_file_path:str = s
        s: str = s.replace(parent_dir + path_sep,"")
    else:
        img_file_path:str = s
    
    if bids_name_dict:
        bids_name_dict: Dict = deepcopy(bids_name_dict)
    else:
        bids_name_dict: Dict = deepcopy(BIDS_PARAM)
    
    mod_found: bool = False
    
    for i in search_arr:
        if mod_found:
            break
        for k,v in i.items():
            if depth(i) == 3:
                for k2,v2 in v.items():
                    mod_type: str = k
                    mod_label: str = k2
                    mod_task: str = ""
                    mod_search: List[str] = v2
                    if list_in_substr(in_list=mod_search,in_str=s):
                        mod_found: bool = True
                        modality_type: str = mod_type
                        modality_label: str = mod_label
                        task: str = mod_task
                        bids_name_dict: Dict = search_bids(s=s,
                                                           bids_search=bids_search,
                                                           bids_map=bids_map,
                                                           modality_type=modality_type,
                                                           modality_label=modality_label,
                                                           bids_name_dict=bids_name_dict)
                            
            elif depth(i) == 4:
                for k2,v2 in v.items():
                    for k3,v3 in v2.items():
                        mod_type: str = k
                        mod_label: str = k2
                        mod_task: str = k3
                        mod_search: List[str] = v3
                        if list_in_substr(in_list=mod_search,in_str=s):
                            mod_found: bool = True
                            modality_type: str = mod_type
                            modality_label: str = mod_label
                            task: str = mod_task
                            bids_name_dict: Dict = search_bids(s=s,
                                                               bids_search=bids_search,
                                                               bids_map=bids_map,
                                                               modality_type=modality_type,
                                                               modality_label=modality_label,
                                                               task=task,
                                                               bids_name_dict=bids_name_dict)

    # Contingency search if initially unsuccessful
    if mod_found:
        pass
    else:
        [modality_type, modality_label, task] = header_search(img_file=img_file_path,
                                                              search_dict=search_dict)
        # Back track if modality type and label were found
        if modality_type and modality_label:
            [bids_name_dict,
             modality_type,
             modality_label,
             task] = bids_id(s=img_file_path,
                             search_dict=search_dict,
                             bids_search=bids_search,
                             bids_map=bids_map,
                             bids_name_dict=bids_name_dict,
                             parent_dir=parent_dir,
                             modality_type=modality_type,
                             modality_label=modality_label,
                             task=task)
    
    return (bids_name_dict,
            modality_type,
            modality_label,
            task)

In [16]:
def _gather_bids_name_args(bids_name_dict: Dict,
                           modality_type: str,
                           param: str
                          ) -> str:
    '''Gathers BIDS naming description arguments for the `construct_bids_name` function.
    
    Usage example:
        >>> param_name = _gather_bids_name_args(bids_dict,
        ...                                     "anat",
        ...                                     "ce")
        ...
        
    Arguments:
        bids_name_dict: BIDS name description dictionary.
        modality_type: Modality type.
        param: BIDS parameter description.
    
    Returns:
        string from the BIDS name description dictionary if it exists, or an empty string otherwise.
    '''
    try:
        if modality_type.lower() == 'fmap' and (param == "case1" or param == "mag2" or param == "case2" or param == "case3" or param == "case4"):
            if param == 'mag2':
                if bids_name_dict[modality_type][param]['magnitude2']:
                    return True
            elif param == 'case4':
                if bids_name_dict[modality_type]["ce"] or bids_name_dict[modality_type]["dir"]:
                    return True
                else:
                    return False
            elif bids_name_dict[modality_type][param]:
                return True
        else:
            return bids_name_dict[modality_type][param]
    except KeyError:
        if param == "case1" or param == "mag2" or param == "case2" or param == "case3" or param == "case4":
            return False
        else:
            return ""

In [17]:
def _get_bids_name_args(bids_name_dict: Dict,
                        modality_type: str
                        ) -> Tuple[str,bool]:
    '''Helper function that wraps the helper funciton `_gather_bids_name_args`.
    
    Usage example:
    Arguments:
        bids_name_dict: BIDS name description dictionary.
        modality_type: Modality type.

    Returns:
        Tuple of strings and booleans that represent:
            * task: str, task name BIDS filename description.
            * acq: str, acquisition description.
            * ce: str, contrast enhanced description
            * acq_dir: str, acquisition direction description.
            * rec: str, reconstruction methods description.
            * echo: str, echo number description from multi-echo acquisition.
            * case1: bool, fieldmap BIDS case 1.
            * mag2: bool, fieldmap BIDS case 1, that includes 2nd magnitude image.
            * case2: bool, fieldmap BIDS case 2.
            * case3: bool, fieldmap BIDS case 3.
            * case4: bool, fieldmap BIDS case 4.
    '''
    task: str = ""
    acq: str = ""
    ce: str = ""
    acq_dir: str = ""
    rec: str = ""
    echo: str = ""
    case1: bool = False
    mag2: bool = False
    case2: bool = False
    case3: bool = False
    case4: bool = False

    params_var: List[str] = [task, acq, ce, acq_dir, rec, echo, case1, mag2, case2, case3, case4]
    params_str: List[str] = ["task", "acq", "ce", "acq_dir", "rec", "echo", "case1", "mag2", "case2", "case3", "case4"]

    if len(params_str) == len(params_var):
        pass
    else:
        raise IndexError("Paired arrays of two different lengths are being compared.")

    for i in range(0,len(params_var)):
        params_var[i] = _gather_bids_name_args(bids_name_dict=bids_name_dict,
                                               modality_type=modality_type,
                                               param=params_str[i])

    return tuple(params_var)

In [18]:
def _make_bids_name(bids_name_dict: Dict,
                    modality_type: str
                   ) -> Tuple[str,str,str,str]:
    '''Helper funciton that creates a BIDS compliant filename given a BIDS name description dictionary,
    and the modality type.

    Usage example:
        >>> _make_bids_name(bids_name_dict=bids_dict,
        ...                 modality_type="anat")
        ...
        "sub-001_ses-001_run-01_T1w"
        
    Arguments:
        bids_name_dict: BIDS name description dictionary.
        modality_type: Modality type.

    Returns:
        BIDS compliant filename string.
    '''
    
    bids_keys: List[str] = list(bids_name_dict[modality_type].keys())
    
    sub: str = bids_name_dict['info']['sub']
    ses: str = bids_name_dict['info']['ses']
    run: Union[int,str] = bids_name_dict[modality_type]['run']

    [task, 
     acq, 
     ce, 
     acq_dir, 
     rec, 
     echo, 
     case1, 
     mag2, 
     case2, 
     case3, 
     case4] = _get_bids_name_args(bids_name_dict=bids_name_dict,
                                  modality_type=modality_type)
    
    f_name: str = ""
    f_name += f"sub-{sub}"
    
    if ses:
        f_name += f"_ses-{ses}"
    
    if task and ('task' in bids_keys):
        f_name += f"_task-{task}"
    
    if acq and ('acq' in bids_keys):
        f_name += f"_acq-{acq}"
    
    if ce and ('ce' in bids_keys):
        f_name += f"_ce-{ce}"
    
    if acq_dir and ('dir' in bids_keys):
        f_name += f"_dir-{acq_dir}"
    
    if rec and ('rec' in bids_keys):
        f_name += f"_rec-{rec}"
    
    f_name += f"_run-{run}"
    
    if modality_type.lower() == 'fmap':
        if case1:
            f_name1: str = f_name + "_phasediff"
            f_name2: str = f_name + "_magnitude1"
            return (f_name1,
                   f_name2,
                   "",
                   "")
        elif case1 and mag2:
            f_name1: str = f_name + "_phasediff"
            f_name2: str = f_name + "_magnitude1"
            f_name3: str = f_name + "_magnitude2"
            return (f_name1,
                   f_name2,
                   f_name3,
                   "")
        elif case2:
            f_name1: str = f_name + "_phase1"
            f_name2: str = f_name + "_phase2"
            f_name3: str = f_name + "_magnitude1"
            f_name4: str = f_name + "_magnitude2"
            return (f_name1,
                   f_name2,
                   f_name3,
                   f_name4)
        elif case3:
            f_name1: str = f_name + "_magnitude"
            f_name2: str = f_name + "_fieldmap"
            return (f_name1,
                   f_name2,
                   "",
                   "")
    else:
        modality_label = bids_name_dict[modality_type]['modality_label']
        f_name += f"_{modality_label}"
        return f_name,"","",""

In [19]:
def data_to_bids(sub_data: SubDataInfo,
                 bids_name_dict: Dict,
                 out_dir: str,
                 modality_type: Optional[str] = "",
                 modality_label: Optional[str] = "",
                 task: Optional[str] = "",
                 meta_dict: Optional[Dict] = {},
                 mod_dict: Optional[Dict] = {},
                 log: Optional[LogFile] = None,
                 gzip: bool = True,
                 env: Optional[Dict] = {},
                 dryrun: bool = False,
                 append_dwi_info: bool = True
                 ) -> Tuple[List[str],List[str],List[str],List[str]]:
    '''Converts source data to BIDS named raw data.
    
    Usage example:
    Arguments:
    Returns:
        Tuple of lists that contains:
            * List of image data file(s). Empty string is returned if this file does not exist.
            * List of corresponding JSON (sidecar) file(s). Empty string is returned if this file does not exist.
            * List of corresponding FSL-style bval file(s). Empty string is returned if this file does not exist.
            * List of corresponding FSL-style bvec file(s). Empty string is returned if this file does not exist.
    
    TODO: 
        * Construct dcm2niix's args as a dict, and pass that as args to this function.
        * Init subject log file here.
        * Write documentation.
        * Write log files for each subject.
            * BUG: logging class only writes to one file.
    '''
    sub: Union[int,str] = sub_data.sub
    ses: Union[int,str] = sub_data.ses
    data: str = sub_data.data
    
    sub_dir: str = os.path.join(out_dir,"sub-" + sub)
    sub_tmp: str = sub_dir
    
    if ses:
        sub_dir: str = os.path.join(sub_dir,"ses-" + ses)
    
    if sub_dir:
        sub_dir: str = os.path.abspath(sub_dir)
    else:
        os.makedirs(sub_dir)
        sub_dir: str = os.path.abspath(sub_dir)
    
    out_data_dir: str = os.path.join(sub_dir, modality_type)
    
    # Gather BIDS name description args
    [task, 
     acq, 
     ce, 
     acq_dir, 
     rec, 
     echo, 
     case1, 
     mag2, 
     case2, 
     case3, 
     case4] = _get_bids_name_args(bids_name_dict=bids_name_dict,
                                  modality_type=modality_type)
    
    if ('.par' in data) or ('.dcm' in data) and modality_type and modality_label:
        # Handle source data cases here
        with TmpDir(tmp_dir=sub_tmp,use_cwd=False) as tmp:
            with TmpDir.TmpFile(tmp_dir=tmp.tmp_dir) as f:
                tmp.mk_tmp_dir()
                [_path, basename, _ext] = f.file_parts()
                try:
                    img_data = convert_image_data(file=data,
                                                  basename=basename,
                                                  out_dir=tmp.tmp_dir,
                                                  log=log,
                                                  env=env,
                                                  dryrun=dryrun,
                                                  return_obj=True)

                    # Update JSON files
                    for i in range(0,len(img_data.imgs)):
                        if img_data.jsons[i]:
                            json_dict: Dict = read_json(json_file=img_data.jsons[i])

                            param_dict: Dict = get_data_params(file=data,
                                                               json_file=img_data.jsons[i])

                            metadata: Dict = dict_multi_update(dictionary=None,
                                                               **meta_dict,
                                                               **param_dict,
                                                               **mod_dict)

                            bids_dict: Dict = construct_bids_dict(meta_dict=metadata,
                                                                  json_dict=json_dict)

                            img_data.jsons[i]: str = write_josn(json_file=img_data.jsons[i],
                                                                dictionary=bids_dict)

                            if (modality_type.lower() == 'dwi' or modality_label.lower() == 'dwi') and append_dwi_info:
                                bvals: List[int] = get_bvals(img_data.bvals[i])
                                echo_time: int = bids_dict["EchoTime"] # Not sure if this value will be in sec. or msec.
                                _label: str = ""
                                for bval in bvals:
                                    if bval != 0:
                                        _label += f"b{bval}"
                                _label += f"TE{echo_time}"
                                if acq:
                                    acq += _label
                                else:
                                    acq = _label
                    
                    # BIDS 'fmap' cases
                    case1: bool = False
                    case2: bool = False
                    case3: bool = False
                    case4: bool = False
                    mag2: bool = False
                    
                    if modality_type.lower() == 'fmap':
                        if len(img_data.imgs) == 4:
                            case2: bool = True
                        elif len(img_data.imgs) == 3:
                            case1: bool = True
                            mag2: bool = True
                        elif len(img_data.imgs) == 1:
                            case4 = True
                        elif len(img_data.imgs) == 2:
                            for i in img_data.imgs:
                                # This needs more review, need to know output of fieldmaps from dcm2niix
                                if list_in_substr(['mag','map','a'],i):
                                    case3: bool = True
                                    break
                                else:
                                    case1: bool = True
                        
                    out_data_dir: str = os.path.join(sub_dir, modality_type)
                    
                    if os.path.exists(out_data_dir):
                        pass
                    else:
                        os.path.makedirs(out_data_dir)
                    
                    if modality_type.lower() == 'dwi' or modality_type.lower() == 'func' :
                        num_frames = get_num_frames(img_data.imgs[0])
                        if num_frames == 1:
                            modality_label: str = "sbref"
                    
                    # Re-write BIDS name dictionary and update to reflect NIFTI data
                    bids_name_dict: Dict = construct_bids_name(sub_data=sub_data,
                                                               modality_type=modality_type,
                                                               modality_label=modality_label,
                                                               acq=acq,
                                                               ce=ce,
                                                               task=task,
                                                               acq_dir=acq_dir,
                                                               rec=rec,
                                                               echo=echo,
                                                               case1=case1,
                                                               mag2=mag2,
                                                               case2=case2,
                                                               case3=case3,
                                                               case4=case4,
                                                               zeropad=zeropad,
                                                               out_dir=out_data_dir)
                    
                    [bids_1, bids_2, bids_3, bids_4] = _make_bids_name(bids_name_dict=bids_name_dict,
                                                                       modality_type=modality_type)

                    bids_names: List[str] = [bids_1, bids_2, bids_3, bids_4]
                    
                    # Image data return lists
                    imgs: List[str] = []
                    jsons: List[str] = []
                    bvals: List[str] = []
                    bvecs: List[str] = []
                    
                    for i in range(0,len(img_data.imgs)):
                        out_name: str = ""
                        if gzip:
                            ext: str = ".nii.gz"
                        else:
                            ext: str = ".nii"
                        
                        out_name: str = os.path.join(out_data_dir,bids_names[i])

                        out_nii: str = out_name + ext
                        out_json: str = out_name + ".json"
                        out_bval: str = out_name + ".bval"
                        out_bvec: str = out_name + ".bvec"

                        out_nii = copy(img_data.imgs[i],out_nii)
                        imgs.append(out_nii)

                        if img_data.jsons[i]:
                            out_json = copy(img_data.jsons[i],out_json)
                            jsons.append(out_json)
                        else:
                            jsons.append("")
                        
                        if img_data.bvals[i]:
                            out_bval = copy(img_data.bvals[i],out_bval)
                            bvals.append(out_bval)
                        else:
                            bvals.append("")
                        
                        if img_data.bvecs[i]:
                            out_bvec = copy(img_data.bvecs[i],out_bvec)
                            bvecs.append(out_bvec)
                        else:
                            bvecs.append("")
                    # Clean-up
                    tmp.rm_tmp_dir()
                    return (imgs,
                            jsons,
                            bvals,
                            bvecs)
                except ConversionError:
                    tmp.rm_tmp_dir()
                    return [""],[""],[""],[""]
    elif ('.nii' in data) and modality_type and modality_label:
        # Handle NIFTI cases here
        out_data_dir: str = os.path.join(sub_dir, modality_type)
        with NiiFile(data) as n:
            [path, basename, ext] = n.file_parts()
            img_files: List[str] = glob.glob(os.path.join(path,basename + "*" + ext))
            json_file: str = os.path.join(path,basename + "*.json")
            bval_file: str = ''.join(glob.glob(os.path.join(path,basename + "*.bval*")))
            bvec_file: str = ''.join(glob.glob(os.path.join(path,basename + "*.bvec*")))
            if json_file:
                json_dict: Dict = read_json(json_file=json_file)

                param_dict: Dict = get_data_params(file=data,
                                                   json_file=json_file)

                metadata: Dict = dict_multi_update(dictionary=None,
                                                   **meta_dict,
                                                   **param_dict,
                                                   **mod_dict)

                bids_dict: Dict = construct_bids_dict(meta_dict=metadata,
                                                      json_dict=json_dict)
            else:
                json_dict: Dict = read_json()
                metadata: Dict = dict_multi_update(dictionary=None,
                                                   **meta_dict,
                                                   **mod_dict)
                
                bids_dict: Dict = construct_bids_dict(meta_dict=metadata,
                                                      json_dict=json_dict)

            # BIDS 'fmap' cases
            case1: bool = False
            case2: bool = False
            case3: bool = False
            case4: bool = False
            mag2: bool = False
            
            if modality_type.lower() == 'fmap':
                if len(img_files) == 4:
                    case2: bool = True
                elif len(img_files) == 3:
                    case1: bool = True
                    mag2: bool = True
                elif len(img_files) == 1:
                    case4 = True
                elif len(img_files) == 2:
                    for i in img_data.imgs:
                        # This needs more review, need to know output of fieldmaps from dcm2niix
                        if list_in_substr(['mag','map','a'],i):
                            case3: bool = True
                            break
                        else:
                            case1: bool = True
            
            if modality_type.lower() == 'dwi' or modality_type.lower() == 'func' :
                num_frames = get_num_frames(img_files[0])
                if num_frames == 1:
                    modality_label: str = "sbref"

            if (modality_type.lower() == 'dwi' or modality_label.lower() == 'dwi') and append_dwi_info:
                bvals: List[int] = get_bvals(bval_file)
                echo_time: Union[int,str] = bids_dict["EchoTime"] # Not sure if this value will be in sec. or msec.
                _label: str = ""
                for bval in bvals:
                    if bval != 0:
                        _label += f"b{bval}"
                if echo_time:
                    _label += f"TE{echo_time}"
                if acq:
                    acq += _label
                else:
                    acq = _label

            # Re-write BIDS name dictionary and update to reflect NIFTI data
            bids_name_dict: Dict = construct_bids_name(sub_data=sub_data,
                                                        modality_type=modality_type,
                                                        modality_label=modality_label,
                                                        acq=acq,
                                                        ce=ce,
                                                        task=task,
                                                        acq_dir=acq_dir,
                                                        rec=rec,
                                                        echo=echo,
                                                        case1=case1,
                                                        mag2=mag2,
                                                        case2=case2,
                                                        case3=case3,
                                                        case4=case4,
                                                        zeropad=zeropad,
                                                        out_dir=out_data_dir)
            
            [bids_1, bids_2, bids_3, bids_4] = _make_bids_name(bids_name_dict=bids_name_dict,
                                                               modality_type=modality_type)
            
            bids_names: List[str] = [bids_1, bids_2, bids_3, bids_4]
            
            # Image data return lists
            imgs: List[str] = []
            jsons: List[str] = []
            bvals: List[str] = []
            bvecs: List[str] = []

            if os.path.exists(out_data_dir):
                pass
            else:
                os.path.makedirs(out_data_dir)
            
            for i in range(0,len(img_files)):
                out_name: str = os.path.join(out_data_dir,bids_names[i])

                out_nii: str = out_name + ext
                out_json: str = out_name + ".json"
                out_bval: str = out_name + ".bval"
                out_bvec: str = out_name + ".bvec"

                out_nii = copy(img_files[i],out_nii)
                out_json = write_josn(json_file=out_json,
                                      dictionary=bids_dict)
                
                imgs.append(out_nii)
                jsons.append(out_json)
                
                if bval_file and bvec_file:
                    out_bval = copy(bval_file,out_bval)
                    out_bvec = copy(bvec_file,out_bvec)
                    bvals.append(out_bval)
                    bvecs.append(out_bvec)
                else:
                    out_bval: str = ""
                    out_bvec: str = ""
                    bvals.append("")
                    bvecs.append("")
        
            return (imgs,
                    jsons,
                    bvals,
                    bvecs)
    elif ('.par' in data) or ('.dcm' in data):
        # Handle source data cases here
        with TmpDir(tmp_dir=sub_tmp,use_cwd=False) as tmp:
            with TmpDir.TmpFile(tmp_dir=tmp.tmp_dir) as f:
                tmp.mk_tmp_dir()
                [_path, basename, _ext] = f.file_parts()
                try:
                    img_data = convert_image_data(file=data,
                                                  basename=basename,
                                                  out_dir=tmp.tmp_dir,
                                                  log=log,
                                                  env=env,
                                                  dryrun=dryrun,
                                                  return_obj=True)

                    # Update JSON files
                    for i in range(0,len(img_data.imgs)):
                        if img_data.jsons[i]:
                            json_dict: Dict = read_json(json_file=img_data.jsons[i])

                            param_dict: Dict = get_data_params(file=data,
                                                               json_file=img_data.jsons[i])

                            metadata: Dict = dict_multi_update(dictionary=None,
                                                               **meta_dict,
                                                               **param_dict,
                                                               **mod_dict)

                            bids_dict: Dict = construct_bids_dict(meta_dict=metadata,
                                                                  json_dict=json_dict)

                            img_data.jsons[i]: str = write_josn(json_file=img_data.jsons[i],
                                                                dictionary=bids_dict)
                    
                    out_data_dir: str = os.path.join(out_dir, "unknown")
                    
                    # Re-write BIDS name dictionary and update to reflect NIFTI data
                    bids_name_dict: Dict = construct_bids_name(sub_data=sub_data,
                                                               modality_type=modality_type,
                                                               modality_label=modality_label,
                                                               acq=acq,
                                                               ce=ce,
                                                               task=task,
                                                               acq_dir=acq_dir,
                                                               rec=rec,
                                                               echo=echo,
                                                               case1=case1,
                                                               mag2=mag2,
                                                               case2=case2,
                                                               case3=case3,
                                                               case4=case4,
                                                               zeropad=zeropad,
                                                               out_dir=out_data_dir)
                    
                    # Ensure that DWI and Fieldmap files are not kept as unknown
                    # by back-tracking if that is the case.
                    if img_data.bvals[0] and img_data.bvecs[0]:
                        modality_type = 'dwi'
                        modality_label = 'dwi'
                        sub_data.data = img_data.imgs[0]
                        [imgs, jsons, bvals, bvecs] = data_to_bids(sub_data=sub_data,
                                                                   bids_name_dict=bids_name_dict,
                                                                   out_dir=out_dir,
                                                                   modality_type=modality_type,
                                                                   modality_label=modality_label,
                                                                   meta_dict=meta_dict,
                                                                   mod_dict=mod_dict)
                        return (imgs,
                                jsons,
                                bvals,
                                bvecs)
                    elif len(img_data.imgs) >= 2:
                        modality_type = 'fmap'
                        modality_label = 'fmap'
                        sub_data.data = img_data.imgs[0]
                        [imgs, jsons, bvals, bvecs] = data_to_bids(sub_data=sub_data,
                                                                   bids_name_dict=bids_name_dict,
                                                                   out_dir=out_dir,
                                                                   modality_type=modality_type,
                                                                   modality_label=modality_label,
                                                                   meta_dict=meta_dict,
                                                                   mod_dict=mod_dict)
                        return (imgs,
                                jsons,
                                bvals,
                                bvecs)
                    
                    if os.path.exists(out_data_dir):
                        pass
                    else:
                        os.path.makedirs(out_data_dir)
                    
                    modality_type: str = "unknown"
                    [bids_1, bids_2, bids_3, bids_4] = _make_bids_name(bids_name_dict=bids_name_dict,
                                                                       modality_type=modality_type)

                    bids_names: List[str] = [bids_1, bids_2, bids_3, bids_4]
                    
                    # Image data return lists
                    imgs: List[str] = []
                    jsons: List[str] = []
                    bvals: List[str] = []
                    bvecs: List[str] = []
                    
                    for i in range(0,len(img_data.imgs)):
                        out_name: str = ""
                        if gzip:
                            ext: str = ".nii.gz"
                        else:
                            ext: str = ".nii"
                        
                        out_name: str = os.path.join(out_data_dir,bids_names[i])

                        out_nii: str = out_name + ext
                        out_json: str = out_name + ".json"
                        out_bval: str = out_name + ".bval"
                        out_bvec: str = out_name + ".bvec"

                        out_nii = copy(img_data.imgs[i],out_nii)
                        imgs.append(out_nii)

                        if img_data.jsons[i]:
                            out_json = copy(img_data.jsons[i],out_json)
                            jsons.append(out_json)
                        else:
                            jsons.append("")
                        
                        if img_data.bvals[i]:
                            out_bval = copy(img_data.bvals[i],out_bval)
                            bvals.append(out_bval)
                        else:
                            bvals.append("")
                        
                        if img_data.bvecs[i]:
                            out_bvec = copy(img_data.bvecs[i],out_bvec)
                            bvecs.append(out_bvec)
                        else:
                            bvecs.append("")
                    # Clean-up
                    tmp.rm_tmp_dir()
                    return (imgs,
                            jsons,
                            bvals,
                            bvecs)
                except ConversionError:
                    tmp.rm_tmp_dir()
                    return [""],[""],[""],[""]
    elif ('.nii' in data):
        # Handle NIFTI cases here
        out_data_dir: str = os.path.join(out_dir, "unknown")
        with NiiFile(data) as n:
            [path, basename, ext] = n.file_parts()
            img_files: List[str] = glob.glob(os.path.join(path,basename + "*" + ext))
            json_file: str = os.path.join(path,basename + "*.json")
            bval_file: str = ""
            bvec_file: str = ""
            
            if json_file:
                json_dict: Dict = read_json(json_file=json_file)

                param_dict: Dict = get_data_params(file=data,
                                                    json_file=json_file)

                metadata: Dict = dict_multi_update(dictionary=None,
                                                    **meta_dict,
                                                    **param_dict,
                                                    **mod_dict)

                bids_dict: Dict = construct_bids_dict(meta_dict=metadata,
                                                        json_dict=json_dict)
            else:
                json_dict: Dict = read_json()
                metadata: Dict = dict_multi_update(dictionary=None,
                                                    **meta_dict,
                                                    **mod_dict)
                
                bids_dict: Dict = construct_bids_dict(meta_dict=metadata,
                                                        json_dict=json_dict)


            # Re-write BIDS name dictionary and update to reflect NIFTI data
            bids_name_dict: Dict = construct_bids_name(sub_data=sub_data,
                                                        modality_type=modality_type,
                                                        modality_label=modality_label,
                                                        acq=acq,
                                                        ce=ce,
                                                        task=task,
                                                        acq_dir=acq_dir,
                                                        rec=rec,
                                                        echo=echo,
                                                        case1=case1,
                                                        mag2=mag2,
                                                        case2=case2,
                                                        case3=case3,
                                                        case4=case4,
                                                        zeropad=zeropad,
                                                        out_dir=out_data_dir)
            
            modality_type: str = "unknown"
            [bids_1, bids_2, bids_3, bids_4] = _make_bids_name(bids_name_dict=bids_name_dict,
                                                                modality_type=modality_type)

            if os.path.exists(out_data_dir):
                pass
            else:
                os.path.makedirs(out_data_dir)
            
            out_name: str = os.path.join(out_data_dir,bids_1)
            
            out_nii: str = out_name + ext
            out_json: str = out_name + ".json"

            out_nii = copy(img_files[0],out_nii)
            out_json = write_josn(json_file=out_json,
                                    dictionary=bids_dict)
            if bval_file and bvec_file:
                out_bval = copy(bval_file,out_bval)
                out_bvec = copy(bvec_file,out_bvec)
            else:
                out_bval: str = ""
                out_bvec: str = ""
        
            return ([out_nii],
                    [out_json],
                    [out_bval],
                    [out_bvec])
    else:
        return [""],[""],[""],[""]

In [27]:
def batch_proc(config_file: str,
              study_img_dir: str,
              out_dir: str
              ) -> List[BIDSImgData]:
    '''working doc-string'''
    [search_dict,
     bids_search,
     bids_map,
     meta_dict,
     exclusion_list] = read_config(config_file=config_file,
                                   verbose=verbose)
    
    # Check BIDS searcn and map dictionaries
    if comp_dict(d1=bids_search,d2=bids_map):
        pass
    
    subs_data: List[SubDataInfo] = collect_info(parent_dir=study_img_dir,
                                                exclusion_list=exclusion_list)
    
    subs_bids: List = []
       
    for sub_data in subs_data:
        data: str = sub_data.data
        bids_name_dict: Dict = deepcopy(BIDS_PARAM)
        bids_name_dict['info']['sub'] = sub_data.sub
        if sub_data.ses:
            bids_name_dict['info']['ses'] = sub_data.ses
        [bids_name_dict, 
         modality_type, 
         modality_label, 
         task] = bids_id(s=data,
                         search_dict=search_dict,
                         bids_search=bids_search,
                         bids_map=bids_map,
                         bids_name_dict=bids_name_dict,
                         parent_dir=study_img_dir)
        [meta_com_dict, 
         meta_scan_dict] = get_metadata(dictionary=meta_dict,
                                        modality_type=modality_type,
                                        task=task)
        # Convert source data
        [imgs,
         jsons,
         bvals,
         bvecs] = data_to_bids(sub_data=sub_data,
                               out_dir=out_dir,
                               modality_type=modality_type,
                               modality_label=modality_label,
                               task=task,
                               meta_dict=meta_com_dict,
                               mod_dict=meta_scan_dict)
        tmp_img_data_obj = BIDSImgData(imgs=imgs,
                                       jsons=jsons,
                                       bvals=bvals,
                                       bvecs=bvecs)
        subs_bids.append(tmp_img_data_obj)
    return subs_bids

In [26]:
class BIDSImgData(object):
    '''working doc-string'''
    def __init__(self,
                 imgs: Optional[str] = "",
                 jsons: Optional[str] = "",
                 bvals: Optional[str] = "",
                 bvecs: Optional[str] = ""
                ):
        '''working doc-string'''
        if imgs:
            self.imgs: List = [imgs]
        else:
            self.imgs: List = []
        
        if jsons:
            self.jsons: List = [jsons]
        else:
            self.jsons: List = []
        
        if bvals:
            self.bvals: List = [bvals]
        else:
            self.bvals: List = []
                
        if bvecs:
            self.bvecs: List = [bvecs]
        else:
            self.bvecs: List = []
    
    def __repr__(self):
        '''working doc-string'''
        return (str(
                {"imgs":self.imgs,
                "jsons":self.jsons,
                "bvals":self.bvals,
                "bvecs":self.bvecs})
               )

In [38]:
t = BIDSImgData()

In [39]:
t

{'imgs': [], 'jsons': [], 'bvals': [], 'bvecs': []}

In [42]:
t.__

<bound method BIDSImgData.__repr__ of {'imgs': [], 'jsons': [], 'bvals': [], 'bvecs': []}>

In [20]:
tt = LogFile("test.log.txt"); tt.file = "test.log.txt"

In [21]:
tt.touch()

In [22]:
tt2 = LogFile("test.2.log.txt"); tt2.file = "test.2.log.txt"; tt2.touch()

In [23]:
os.listdir(os.getcwd())

['test.2.log.txt',
 'read_header.ipynb',
 'cs_wrap2.ipynb',
 'test.cp_dir',
 'run1.py',
 'test.log.txt',
 'test.img',
 'cs_wrap1.ipynb',
 '.ipynb_checkpoints',
 'tmp_dir_4541']

In [24]:
tt.info("test 1")

test 1
test 1


In [25]:
tt2.info("test 2")

test 2
test 2


In [49]:
tt2.file = "test.2.log.txt"

In [44]:
tt2.abs_path()

'/Users/adebayobraimah/Desktop/projects/convert_source/work_dir/test.2.log.txt'

In [47]:
tt2.touch()

In [36]:
tt.file

''

In [48]:
os.listdir(os.getcwd())

['test.2.log.txt',
 'read_header.ipynb',
 'cs_wrap2.ipynb',
 'test.cp_dir',
 'run1.py',
 'test.log.txt',
 'test.img',
 'cs_wrap1.ipynb',
 '.ipynb_checkpoints',
 'tmp_dir_4541']